<a href="https://colab.research.google.com/github/AndreiDragomir07/COS484_A2_P1/blob/main/Copy_of_A2P1_Named_Entity_Recognition_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Problem 1: HMM for NER
Welcome to the programming portion of the assignment! Each assignment throughout the semester will have a written portion and a programming portion. We will be using [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true), so if you have never used it before, take a quick look through this introduction: [Working with Google Colab](https://docs.google.com/document/d/1LlnXoOblXwW3YX-0yG_5seTXJsb3kRdMMRYqs8Qqum4/edit?usp=sharing).

### Writing Code
Look for the keyword "TODO" and fill in your code in the empty space.
Feel free to add and delete arguments in function signatures, but be careful that you might need to change them in function calls which are already present in the notebook.

### Data preprocessing

In this section we will write code to load data and build the dataset for Named Entity Recognition.

You may inspect the data first before writing the data preprocessing code by looking at the data file: https://princeton-nlp.github.io/cos484/assignments/a2/eng.train. Hints on processing the data:
- You may ignore the lines with "DOCSTART"
- Examples of NER tags include "O", "ORG", "MISC", and it's always in the same position in each line of the data.
- To process numbers more easily, you can replace all digits with 0's (to avoid out-of-vocab words)

You should end up with a list of sentences, where each sentence is represented with a list of words and tags.

Additional, you will want to support the following functions for later:
- Map words and tags to ids (integers)
- Handle unknown words in mapping

In [11]:
import re

class PreprocessData:
    """
    Preprocess the data: build a list of sentences, where each sentence is a list of (word, tag) tuples.
    Also builds word-to-id and tag-to-id mappings.
    """
    def __init__(self, data):
        self.sentences = []
        current_sentence = []
        # set() creates an empty set, which is an unordered collection of unique elements in Python.
        word_set = set()
        tag_set = set()
        for line in data:
            line = line.strip()
            if not line:
                if current_sentence:
                    self.sentences.append(current_sentence)
                    current_sentence = []
                continue
            parts = line.split()
            if len(parts) < 4:
                continue
            word, tag = parts[0], parts[3]
            if word == "-DOCSTART-":
                continue
            word = re.sub(r'\d', '0', word)
            current_sentence.append((word, tag))
            word_set.add(word)
            tag_set.add(tag)
        if current_sentence:
            self.sentences.append(current_sentence)
        # Build word-to-id and tag-to-id dictionaries. Reserve 0 for <UNK> for words.
        self.word2id = {word: idx+1 for idx, word in enumerate(sorted(word_set))}
        self.word2id["<UNK>"] = 0
        self.tag2id = {tag: idx+1 for idx, tag in enumerate(sorted(tag_set))}


In [2]:
import requests

train_data_url = "https://princeton-nlp.github.io/cos484/assignments/a2/eng.train"
response = requests.get(train_data_url)
response.raise_for_status() # Raise an exception for HTTP errors
train_data_lines = response.text.splitlines()

print(f"Successfully downloaded {len(train_data_lines)} lines of training data.")
print("First 5 lines:")
for i in range(min(5, len(train_data_lines))):
    print(train_data_lines[i])

Successfully downloaded 219553 lines of training data.
First 5 lines:
EU NNP I-NP ORG
rejects VBZ I-VP O
German JJ I-NP MISC
call NN I-NP O
to TO I-VP O


In [12]:
processed_train_data = PreprocessData(train_data_lines)

print(f"Number of sentences in training data: {len(processed_train_data.sentences)}")
print(f"Number of unique words in training data: {len(processed_train_data.word2id)}")
print(f"Number of unique tags in training data: {len(processed_train_data.tag2id)}")
print("First sentence (word, tag) tuples:")
print(processed_train_data.sentences[0])
print("First 5 word to ID mappings:")
for i, (word, idx) in enumerate(processed_train_data.word2id.items()):
    if i >= 5: break
    print(f"  {word}: {idx}")
print("First 5 tag to ID mappings:")
for i, (tag, idx) in enumerate(processed_train_data.tag2id.items()):
    if i >= 5: break
    print(f"  {tag}: {idx}")

Number of sentences in training data: 14041
Number of unique words in training data: 20101
Number of unique tags in training data: 5
First sentence (word, tag) tuples:
[('EU', 'ORG'), ('rejects', 'O'), ('German', 'MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'MISC'), ('lamb', 'O'), ('.', 'O')]
First 5 word to ID mappings:
  !: 1
  ": 2
  $: 3
  %: 4
  &: 5
First 5 tag to ID mappings:
  LOC: 1
  MISC: 2
  O: 3
  ORG: 4
  PER: 5


### Hidden Markov Model
In this section, we will implement a bigram hidden markov model (HMM) that could perform two types of decoding: greedy decoding and viterbi decoding.

Specifically, you should include the following functionalities:
- Initialize the HMM given the word and tag mappings.
- Train the HMM with a given corpus
- Greedy decoding: given a single sentence, output its tags with greedy algorithm
- Viterbi decoding: given a single sentence, output its tags using Viterbi

You may refer to the lecture notes for more details on the HMM and the decoding algorithms.

In [13]:
import numpy as np

class HMM_Train:
    """
    Trains an HMM by counting tag-to-tag transitions and tag-to-word emissions.
    Sentence start is treated as a void tag at position 0 (transition from void to first tag).
    - transition_counts: (num_tags+1, num_tags+1). Row 0 = counts from void (start of sentence) to each tag.
    - emission_counts: (num_tags+1, num_words); emission_counts[tag_idx][word_id] = count.
    """
    def __init__(self, sentences, word2id, tag2id):
        """
        Args:
            sentences: list of sentences, each sentence is a list of (word, tag) tuples
            word2id: dict mapping word -> int id
            tag2id: dict mapping tag -> int id. Assumes tag IDs start from 1, and 0 is not occupied.
        """
        self.sentences = sentences
        self.word2id = word2id
        self.tag2id = tag2id
        num_tags = len(tag2id)
        num_words = len(word2id)

        # Transition counts: (num_tags + 1, num_tags + 1)
        # Row 0 = from void (start of sentence)
        # Columns 0 = to void (will remain 0), 1..num_tags = to tag id 1..num_tags
        self.transition_counts = np.zeros((num_tags + 1, num_tags + 1), dtype=np.float64)

        # Emission counts: (num_tags + 1, num_words)
        # Row 0 = for void (will remain 0), rows 1..num_tags = for tag id 1..num_tags
        self.emission_counts = np.zeros((num_tags + 1, num_words), dtype=np.float64)

        for sentence in sentences:
            prev_tag_id = 0  # 0 = void (start of sentence)
            for word, tag in sentence:
                word_id = word2id.get(word, 0) # 0 for <UNK>
                tag_id = tag2id[tag] # tag_id is 1-indexed

                # Use 1-indexed tag_id directly for column in transition_counts
                self.transition_counts[prev_tag_id, tag_id] += 1
                # Use 1-indexed tag_id directly for row in emission_counts
                self.emission_counts[tag_id, word_id] += 1
                prev_tag_id = tag_id

    def counts_to_probabilities(self, counts_matrix, k=0.0):
        """
        Converts a count matrix into a probability matrix using add-k smoothing.

        Args:
            counts_matrix (np.ndarray): The matrix of counts.
            k (float): The smoothing parameter (add-k smoothing). Default is 0.0 (no smoothing).

        Returns:
            np.ndarray: The probability matrix.
        """
        if k == 0.0:
            # Avoid division by zero for rows that sum to 0 if no smoothing
            row_sums = np.sum(counts_matrix, axis=1, keepdims=True)
            # Replace 0 sums with 1 to prevent division by zero, resulting in 0 probabilities for that row
            row_sums[row_sums == 0] = 1
            probabilities = counts_matrix / row_sums
        else:
            num_columns = counts_matrix.shape[1]
            smoothed_counts = counts_matrix + k
            denominator = np.sum(counts_matrix, axis=1, keepdims=True) + k * num_columns
            probabilities = smoothed_counts / denominator
        return probabilities


In [14]:
hmm_trainer = HMM_Train(
    processed_train_data.sentences,
    processed_train_data.word2id,
    processed_train_data.tag2id
)

# Calculate transition probabilities (with smoothing)
transition_probabilities = hmm_trainer.counts_to_probabilities(
hmm_trainer.transition_counts, k=0.1
)

# Explicitly set the 'to void' probabilities (column 0) to zero, then re-normalize.
# This assumes that the 'void' state (tag_id = 0) is never a destination.
transition_probabilities[:, 0] = 0.0 # Set all probabilities of transitioning TO void to 0

# Re-normalize rows to ensure probabilities sum to 1 after zeroing.
# This is crucial for valid probabilities for subsequent calculations.
row_sums_after_zeroing = np.sum(transition_probabilities, axis=1, keepdims=True)
# Avoid division by zero for rows that might still sum to 0 after zeroing
row_sums_after_zeroing[row_sums_after_zeroing == 0] = 1
transition_probabilities = transition_probabilities / row_sums_after_zeroing

# Calculate emission probabilities (add-10 smoothing)
emission_probabilities = hmm_trainer.counts_to_probabilities(
hmm_trainer.emission_counts, k=1.0
)

# Modify the following print statement to print out the transition count matrix:
with np.printoptions(suppress=True):
    print("The transition count matrix is the following: \n", hmm_trainer.transition_counts)

print("Shape of transition probabilities matrix:", transition_probabilities.shape)
print("Shape of emission probabilities matrix:", emission_probabilities.shape)

print("\nFull transition probabilities matrix:")
with np.printoptions(suppress=True, threshold=np.inf):
    print(transition_probabilities)

print("\nFirst 5 rows and 5 columns of emission probabilities:")
print(emission_probabilities[:5, :5])

The transition count matrix is the following: 
 [[     0.   1581.    502.   8154.   2455.   1349.]
 [     0.   1168.     30.   6927.     11.      1.]
 [     0.     10.   1190.   3152.     39.     61.]
 [     0.   5533.   2861. 138886.   3792.   5183.]
 [     0.      2.      9.   6105.   3728.      6.]
 [     0.      3.      1.   6354.      0.   4528.]]
Shape of transition probabilities matrix: (6, 6)
Shape of emission probabilities matrix: (6, 20101)

Full transition probabilities matrix:
[[0.         0.11260193 0.03575829 0.58071431 0.17484599 0.09607948]
 [0.         0.14354531 0.00369892 0.85125653 0.00136406 0.00013518]
 [0.         0.00226839 0.26728804 0.70793936 0.00878158 0.01372263]
 [0.         0.03541059 0.0183104  0.88883975 0.02426859 0.03317067]
 [0.         0.00021319 0.00092381 0.61977565 0.3784681  0.00061926]
 [0.         0.00028476 0.00010104 0.58366785 0.00000919 0.41593717]]

First 5 rows and 5 columns of emission probabilities:
[[4.97487687e-05 4.97487687e-05 4.97

In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score

class HMM:
    def __init__(self, word2id, tag2id, transition_probabilities, emission_probabilities):
        self.word2id = word2id
        self.tag2id = tag2id
        self.id2tag = {v: k for k, v in tag2id.items()}
        self.transition_probabilities = transition_probabilities
        self.emission_probabilities = emission_probabilities
        self.num_tags = len(tag2id)

    def greedy_decode(self, sentence):
        predicted_tags_ids = []
        prev_tag_id = 0  # 0 represents the 'void' start state

        for word in sentence:
            word_id = self.word2id.get(word, 0) # 0 for <UNK>

            best_score = -np.inf
            best_tag_id = -1

            for current_tag_idx in range(1, self.num_tags + 1): # Iterate through 1-indexed tag IDs

                # prev_tag_id (0 for void, or 1-indexed tag ID) as row index
                # current_tag_idx (1-indexed tag ID) as column index
                trans_prob = self.transition_probabilities[prev_tag_id, current_tag_idx]

                # current_tag_idx (1-indexed tag ID) as row index
                emit_prob = self.emission_probabilities[current_tag_idx, word_id]

                score = np.log(trans_prob) + np.log(emit_prob)
                if score > best_score:
                    best_score = score
                    best_tag_id = current_tag_idx # current_tag_idx is already 1-indexed

            predicted_tags_ids.append(best_tag_id)
            prev_tag_id = best_tag_id # Update previous tag for next iteration

        return [self.id2tag[tag_id] for tag_id in predicted_tags_ids]

    def viterbi_decode(self, sentence):
        sentence_len = len(sentence)

        # Viterbi matrix: V[tag_id][word_idx] stores the max log probability
        # Backpointer matrix: B[tag_id][word_idx] stores the previous tag_id
        # Using num_tags + 1 for rows to directly map 1-indexed tag_ids.
        V = np.full((self.num_tags + 1, sentence_len), -np.inf)
        B = np.zeros((self.num_tags + 1, sentence_len), dtype=int)

        # --- Initialization (t=0) ---
        first_word_id = self.word2id.get(sentence[0], 0)
        for current_tag_idx in range(1, self.num_tags + 1):
            # From void (state 0) to current tag
            trans_prob = self.transition_probabilities[0, current_tag_idx]
            emit_prob = self.emission_probabilities[current_tag_idx, first_word_id]
            V[current_tag_idx, 0] = np.log(trans_prob) + np.log(emit_prob)
            B[current_tag_idx, 0] = 0 # No previous tag for the first word (from void)

        # --- Recursion (t=1 to sentence_len - 1) ---
        for t in range(1, sentence_len):
            word_id = self.word2id.get(sentence[t], 0)
            for current_tag_idx in range(1, self.num_tags + 1):
                best_prev_score = -np.inf
                best_prev_tag_id = -1
                for prev_tag_idx in range(1, self.num_tags + 1):
                    score = V[prev_tag_idx, t-1] \
                            + np.log(self.transition_probabilities[prev_tag_idx, current_tag_idx]) \
                            + np.log(self.emission_probabilities[current_tag_idx, word_id])

                    if score > best_prev_score:
                        best_prev_score = score
                        best_prev_tag_id = prev_tag_idx
                V[current_tag_idx, t] = best_prev_score
                B[current_tag_idx, t] = best_prev_tag_id

        # --- Termination ---
        # Find the path with the highest probability at the last word
        last_word_col = sentence_len - 1
        best_last_tag_id = np.argmax(V[1:, last_word_col]) + 1 # +1 because V[0,:] is unused for tags
        # max_log_prob = V[best_last_tag_id, last_word_col] # Not explicitly needed for path reconstruction

        # --- Backtracking ---
        predicted_tags_ids = [0] * sentence_len
        predicted_tags_ids[last_word_col] = best_last_tag_id

        for t in range(sentence_len - 1, 0, -1):
            predicted_tags_ids[t-1] = B[predicted_tags_ids[t], t]

        # Convert tag IDs to actual tag names
        return [self.id2tag[tag_id] for tag_id in predicted_tags_ids]

    def evaluate_accuracy(self, test_sentences, decoding_method='viterbi'):
        if decoding_method not in ['greedy', 'viterbi']:
            raise ValueError("decoding_method must be 'greedy' or 'viterbi'")

        correct_predictions = 0
        total_predictions = 0

        for sentence_data in test_sentences:
            words = [item[0] for item in sentence_data]
            true_tags = [item[1] for item in sentence_data]

            if decoding_method == 'greedy':
                predicted_tags = self.greedy_decode(words)
            else:
                predicted_tags = self.viterbi_decode(words)

            for i in range(len(true_tags)):
                total_predictions += 1
                if predicted_tags[i] == true_tags[i]:
                    correct_predictions += 1

        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
        return accuracy

    def compute_confusion_matrix(self, test_sentences, decoding_method='viterbi'):
        if decoding_method not in ['greedy', 'viterbi']:
            raise ValueError("decoding_method must be 'greedy' or 'viterbi'")

        all_true_tags = []
        all_predicted_tags = []

        for sentence_data in test_sentences:
            words = [item[0] for item in sentence_data]
            true_tags = [item[1] for item in sentence_data]

            if decoding_method == 'greedy':
                predicted_tags = self.greedy_decode(words)
            else:
                predicted_tags = self.viterbi_decode(words)

            all_true_tags.extend(true_tags)
            all_predicted_tags.extend(predicted_tags)

        # Get all unique tags from tag2id, sorted by their ID for consistent order
        labels = sorted(self.tag2id.keys(), key=lambda x: self.tag2id[x])

        # Compute confusion matrix
        cm = confusion_matrix(all_true_tags, all_predicted_tags, labels=labels)
        return cm, labels

    def compute_f1_score(self, test_sentences, decoding_method='viterbi', average='weighted'):
        if decoding_method not in ['greedy', 'viterbi']:
            raise ValueError("decoding_method must be 'greedy' or 'viterbi'")

        all_true_tags = []
        all_predicted_tags = []

        for sentence_data in test_sentences:
            words = [item[0] for item in sentence_data]
            true_tags = [item[1] for item in sentence_data]

            if decoding_method == 'greedy':
                predicted_tags = self.greedy_decode(words)
            else:
                predicted_tags = self.viterbi_decode(words)

            all_true_tags.extend(true_tags)
            all_predicted_tags.extend(predicted_tags)

        labels = sorted(self.tag2id.keys(), key=lambda x: self.tag2id[x])

        # Compute F1 score
        f1 = f1_score(all_true_tags, all_predicted_tags, labels=labels, average=average)
        return f1

In [ ]:
# Initialize the HMM model with the calculated probabilities and mappings
hmm_model = HMM(
    processed_train_data.word2id,
    processed_train_data.tag2id,
    transition_probabilities,
    emission_probabilities
)


print("\n--- Evaluating on Training Data ---")

# Evaluate with greedy decoding on full training set
greedy_train_accuracy = hmm_model.evaluate_accuracy(processed_train_data.sentences, decoding_method='greedy')
print(f"Greedy decoding accuracy on training data: {greedy_train_accuracy:.4f}")

# Evaluate with Viterbi decoding on full training set
viterbi_train_accuracy = hmm_model.evaluate_accuracy(processed_train_data.sentences, decoding_method='viterbi')
print(f"Viterbi decoding accuracy on training data: {viterbi_train_accuracy:.4f}")

# Compute Confusion Matrix for greedy decoding on training data
cm_greedy, cm_labels = hmm_model.compute_confusion_matrix(processed_train_data.sentences, decoding_method='greedy')
print(f"\nConfusion Matrix (Greedy, training data):\n{cm_greedy}")
print(f"Confusion Matrix Labels: {cm_labels}")

# Compute F1 Score for greedy decoding on training data
f1_greedy = hmm_model.compute_f1_score(processed_train_data.sentences, decoding_method='greedy')
print(f"F1 Score (Greedy, training data): {f1_greedy:.4f}")

# Compute Confusion Matrix for Viterbi decoding on training data
cm_viterbi, _ = hmm_model.compute_confusion_matrix(processed_train_data.sentences, decoding_method='viterbi')
print(f"\nConfusion Matrix (Viterbi, training data):\n{cm_viterbi}")

# Compute F1 Score for Viterbi decoding on training data
f1_viterbi = hmm_model.compute_f1_score(processed_train_data.sentences, decoding_method='viterbi')
print(f"F1 Score (Viterbi, training data): {f1_viterbi:.4f}")

### Train and evaluate HMMs.

In this section, you will implement the logic for training and evaluating the HMMs:
- Train the model by calling the functions/classes you implemented above,
- Evaluate the trained model on the training and evaluation set by calculating the accuracy of the predicted tags.
- Compute the confusion matrix and F1 score of the predictions.

### Experiments

#### Load data

Download and load the data from the following links

https://princeton-nlp.github.io/cos484/assignments/a2/eng.train

https://princeton-nlp.github.io/cos484/assignments/a2/eng.val

Then load the data using what you implemented

#### Experiment with an HMM with greedy decoding

**(a) Which pair of tags does the model have most difficulty separating according to the confusion matrix of the validation set?**

TODO: ANSWER THE QUESTION HERE (DOUBLE-CLICK TO EDIT)


#### Experiment with an HMM with viterbi decoding

**(b) What major differences do you observe compared to the matrix in (a)**

TODO: ANSWER THE QUESTION HERE (DOUBLE-CLICK TO EDIT)


# LLM Prompts

If you used an AI tool to complete any part of this assignment, please paste all prompts you used to produce your final code/responses in the box below and answer the following reflection question.

Prompts Used:
*   
*   



**Reflection: What parts of the AI generated output required modification or improvement? Describe the feedback you gave the tool to produce your final output or any changes you had to make on your own.**

TODO: ANSWER THE QUESTION HERE (DOUBLE-CLICK TO EDIT)

# Task
Process and evaluate the validation data from "https://princeton-nlp.github.io/cos484/assignments/a2/eng.val" using the `PreprocessData` class and the existing HMM model to calculate and print the accuracy, confusion matrix, and F1 score for both greedy and Viterbi decoding.

## Fetch Validation Data

### Subtask:
Download the content of the 'eng.val' file from 'https://princeton-nlp.github.io/cos484/assignments/a2/eng.val'. This will involve using the `requests` library to fetch the data.


**Reasoning**:
The user wants to download the validation data. I need to use the `requests` library to fetch the data from the specified URL, handle potential errors, and then process the text into lines, finally printing a confirmation.



In [30]:
import requests

val_data_url = "https://princeton-nlp.github.io/cos484/assignments/a2/eng.val"
response = requests.get(val_data_url)
response.raise_for_status() # Raise an exception for HTTP errors
val_data_lines = response.text.splitlines()

print(f"Successfully downloaded {len(val_data_lines)} lines of validation data.")
print("First 5 lines:")
for i in range(min(5, len(val_data_lines))):
    print(val_data_lines[i])

Successfully downloaded 53015 lines of validation data.
First 5 lines:
CRICKET NNP I-NP O
- : O O
LEICESTERSHIRE NNP I-NP ORG
TAKE NNP I-NP O
OVER IN I-PP O


**Reasoning**:
The validation data has been downloaded. Now, I need to preprocess it using the `PreprocessData` class, similar to how the training data was handled. This will convert the raw text lines into a structured format (list of sentences with word-tag tuples) that can be used by the HMM model for evaluation.



In [31]:
processed_val_data = PreprocessData(val_data_lines)

print(f"Number of sentences in validation data: {len(processed_val_data.sentences)}")
print(f"Number of unique words in validation data: {len(processed_val_data.word2id)}")
print(f"Number of unique tags in validation data: {len(processed_val_data.tag2id)}")
print("First sentence (word, tag) tuples from validation data:")
print(processed_val_data.sentences[0])

Number of sentences in validation data: 3490
Number of unique words in validation data: 8697
Number of unique tags in validation data: 5
First sentence (word, tag) tuples from validation data:
[('CRICKET', 'O'), ('-', 'O'), ('LEICESTERSHIRE', 'ORG'), ('TAKE', 'O'), ('OVER', 'O'), ('AT', 'O'), ('TOP', 'O'), ('AFTER', 'O'), ('INNINGS', 'O'), ('VICTORY', 'O'), ('.', 'O')]


**Reasoning**:
Now that the validation data is preprocessed, I will evaluate the pre-trained HMM model on this validation set using both greedy and Viterbi decoding. I need to calculate and print the accuracy, confusion matrix, and F1 score for each decoding method, similar to how it was done for the training data.



In [32]:
print("\n--- Evaluating on Validation Data ---")

# Evaluate with greedy decoding on validation set
greedy_val_accuracy = hmm_model.evaluate_accuracy(processed_val_data.sentences, decoding_method='greedy')
print(f"Greedy decoding accuracy on validation data: {greedy_val_accuracy:.4f}")

# Compute Confusion Matrix for greedy decoding on validation data
cm_greedy_val, cm_labels_val = hmm_model.compute_confusion_matrix(processed_val_data.sentences, decoding_method='greedy')
print(f"\nConfusion Matrix (Greedy, validation data):\n{cm_greedy_val}")
print(f"Confusion Matrix Labels: {cm_labels_val}")

# Compute F1 Score for greedy decoding on validation data
f1_greedy_val = hmm_model.compute_f1_score(processed_val_data.sentences, decoding_method='greedy')
print(f"F1 Score (Greedy, validation data): {f1_greedy_val:.4f}")

# Evaluate with Viterbi decoding on validation set
viterbi_val_accuracy = hmm_model.evaluate_accuracy(processed_val_data.sentences, decoding_method='viterbi')
print(f"\nViterbi decoding accuracy on validation data: {viterbi_val_accuracy:.4f}")

# Compute Confusion Matrix for Viterbi decoding on validation data
cm_viterbi_val, _ = hmm_model.compute_confusion_matrix(processed_val_data.sentences, decoding_method='viterbi')
print(f"\nConfusion Matrix (Viterbi, validation data):\n{cm_viterbi_val}")

# Compute F1 Score for Viterbi decoding on validation data
f1_viterbi_val = hmm_model.compute_f1_score(processed_val_data.sentences, decoding_method='viterbi')
print(f"F1 Score (Viterbi, validation data): {f1_viterbi_val:.4f}")


--- Evaluating on Validation Data ---
Greedy decoding accuracy on validation data: 0.9169

Confusion Matrix (Greedy, validation data):
[[ 1505    22   261   173    14]
 [   58   506   339    98     6]
 [   46    92 40622   379    25]
 [  263    51   712  1176    48]
 [   23    19  1231   217  1200]]
Confusion Matrix Labels: ['LOC', 'MISC', 'O', 'ORG', 'PER']
F1 Score (Greedy, validation data): 0.9096

Viterbi decoding accuracy on validation data: 0.9290

Confusion Matrix (Viterbi, validation data):
[[ 1509    21   217   194    34]
 [   19   652   239    77    20]
 [   52   140 40308   485   179]
 [  145    30   506  1529    40]
 [   20    12   827   228  1603]]
F1 Score (Viterbi, validation data): 0.9269
